# End-to-end DSPy Workflows Guide 

This guide will cover the following topics:

## Creating a Multi-stage LLM Pipeline
- Building a pipeline with an untuned model in DSPy
- Implementing batch inference (using Ray data)

## Improving the Pipeline
1. Prompt optimization
2. Fine-tuning
    - How to make an 8B model perform almost as well as a 70B model in your pipeline
3. Combining fine-tuning with prompt optimization

## Deployment
- Steps to deploy the optimized pipeline and fine-tuned model to production

## Future Work and Open Questions
- Efficient batch inference with a DSPy pipeline
- Exploring different fine-tuning methods and hyperparameter sweeps

This guide aims to provide a comprehensive overview of building, optimizing, and deploying LLM pipelines using DSPy and Anyscale.

## Set up

Node Set up:

We will be running everything on a head node that uses 4xA100-80GB GPUs. I find that L4s are usually available and suitable for this usecase. You can also use any more powerful node.

To change to use A100 GPUs, click the "1 active node" in the top right corner, then for workspace node, click the pencil icon and navigate to the A100 tab and select the 4xA100 option. If you do not see A100 in the list of GPUs, they may not be available on your cloud. Choose another kind of GPU (This notebook has been tested on X, and Y as alternatives) (TODO)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# TODO(work): DSPy installation cell
# TODO(decision): are these changes going to be merged into DSPy main

# TODO: look at my own init file to see all the stupid extra pip installs

# !pip install -e dspy-d
# !pip install -r dspy-d/requirements.txt
# !pip install vllm

# ignore future warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import dspy
import dsp
import os
from dotenv import load_dotenv
# TODO: include cache in notebook
cache_dir = "/home/ray/default/dspy/cache"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
# I have included a .env.example with the necessary environment variables to be set
# You can also set them manually if you prefer

os.environ["DSP_CACHEDIR"] = cache_dir

load_dotenv()

WARNING 09-24 09:57:06 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.
unable to import apply_chat_template


True

In [4]:
necessary_env_vars = [
    "DSP_CACHEDIR",
    "HF_TOKEN",
    "HF_HOME"
]

for var in necessary_env_vars:
    assert os.environ[var], f"{var} is not set"

In [5]:
import ray

if not ray.is_initialized():
    ray.init(runtime_env={"env_vars": os.environ, "py_modules": [dspy, dsp]})

2024-09-24 09:57:10,536	INFO worker.py:1603 -- Connecting to existing Ray cluster at address: 10.0.15.220:6379...
2024-09-24 09:57:10,544	INFO worker.py:1779 -- Connected to Ray cluster. View the dashboard at https://session-fkvdirx4bzefi53sjl55m7asad.i.anyscaleuserdata.com 
2024-09-24 09:57:10,566	INFO packaging.py:530 -- Creating a file package for local directory '/home/ray/default/dspy/dspy'.
2024-09-24 09:57:10,587	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_8ace7ac78ed16883.zip' (0.78MiB) to Ray cluster...
2024-09-24 09:57:10,591	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_8ace7ac78ed16883.zip'.
2024-09-24 09:57:10,603	INFO packaging.py:530 -- Creating a file package for local directory '/home/ray/default/dspy/dsp'.
2024-09-24 09:57:10,622	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_956d9051ac54fdb1.zip' (0.86MiB) to Ray cluster...
2024-09-24 09:57:10,625	INFO packaging.py:371 -- Successfully pushed file package 'g

(autoscaler +1m41s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


We will make use of a random number generator in this notebook. We are creating a Random object here to ensure that our notebook is reproducible.

In [6]:
import random

rng = random.Random()

# Creating your multi-stage LLM pipeline

In [7]:
from dspy.datasets import HotPotQA
from dspy.evaluate import Evaluate
from dsp.utils.utils import deduplicate


# We are setting the experimental flag to True to make use of the fine-tuning
# features that are still in development.
dspy.settings.configure(experimental=True)

# Define the program
class BasicMH(dspy.Module):
    def __init__(self, passages_per_hop=3, num_hops=2):
        super().__init__()
        self.num_hops = num_hops
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        # self.generate_query = dspy.ChainOfThought(GenerateSearchQuery)
        self.generate_query = [dspy.ChainOfThought("context, question -> search_query") for _ in range(self.num_hops)]
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        context = []
        
        for hop in range(self.num_hops):
            search_query = self.generate_query[hop](context=context, question=question).search_query
            passages = self.retrieve(search_query).passages
            context = deduplicate(context + passages)

        prediction = self.generate_answer(context=context, question=question).copy(context=context)
        return prediction

# Let's break down the BasicMH program

## Program Design
The BasicMH program is designed to answer complex questions requiring multiple retrieval steps without relying on the model's internal knowledge.

For example, if we asked "What the capital of the country that has the eiffel tower"

The model cannot just throw the original question "What the capital of the country that has the eiffel tower" into a retriever, the model needs to generate a query with the right search terms in order to get the correct passages.

### Program Flow:
1. Iterate through the specified number of hops:

   a. Generate a search query using `generate_query` modules.

   b. Retrieve passages using the `retrieve` module.

   c. Deduplicate and add retrieved passages to the context.
2. After completing all hops, generate the final answer using the `generate_answer` module.

This multi-step approach enables the program to break down complex questions and gather necessary information incrementally, improving the accuracy of the final answer.

## Program Implementation

### `BasicMH` class

#### `__init__` method

This method initializes the class with the specified number of hops and passages per hop. It also sets up the `retrieve` module for passage retrieval and a list of `generate_query` modules for generating search queries at each hop. It should be noted that each hop has a different generate_query module, which means they all get optimized separately. The intuition here is that a good prompt for the first hop may not be good for the second hop, and so on.

#### `forward` method

This method implements the forward pass of the program. It iterates through the specified number of hops, generating a search query at each hop and retrieving passages. The retrieved passages are then deduplicated and added to the context. After completing all hops, it generates the final answer using the `generate_answer` module.

A note here is that you directly return the result of the `generate_answer` module. A common anti-pattern is to return `prediction.answer` instead of `prediction`. This is because `prediction` is an object that contains information that is useful for optimization, and the answer can be extracted later.

### Example Usage

Here's an example of how to use the `BasicMH` class:

```
program = BasicMH()
prediction = program(question="What is the capital of the country that has the Eiffel Tower?")
print(prediction.answer)
# Prints: Paris
```

Below we load the dataset using a built in `HotPotQA` dataset class from DSPy.

We set the `train_seed` and `eval_seed` to `0` for reproducibility and the `test_size` to `0` because we do not need a test set for this tutorial.

In [8]:
# Prepare the dataset
TRAIN_SIZE = 1500
DEV_SIZE = 1500
dataset = HotPotQA(train_seed=0, eval_seed=0, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Here we set up the metric and evaluator. We will be using the answer exact match metric.

The evaluator is what we will consider as our test set.

We choose `num_threads=90` because we are bottlenecked by the retrieval server, and through testing this is the maximum number of concurrent threads that can be run without causing issues for other people using the retrieval server.

In [9]:
# Prepare the metric and evaluator
NUM_THREADS = 90
metric = dspy.evaluate.answer_exact_match
evaluate_devset = Evaluate(devset=devset[:DEV_SIZE], metric=metric, num_threads=NUM_THREADS, display_progress=True)

TODO(optional): Implement LLM as judge

In [10]:
# Prepare the retriever model
# Note that this is not hosted on Anyscale
COLBERT_V2_ENDPOINT = "http://20.102.90.50:2017/wiki17_abstracts"
retriever = dspy.ColBERTv2(url=COLBERT_V2_ENDPOINT)

## Gathering baseline performance

run evaluate on a base pipeline

In [11]:
MAX_TOKENS = 3000
MODEL_PARAMETERS = {
  "max_tokens": MAX_TOKENS,
  "temperature": 0,
}

LOCAL_API_PARAMETERS = {
  "api_base": "http://localhost:8000/v1",
  "api_provider": "vllm",
  "api_key": "fake-key-doesnt-matter"
}
vanilla_program = BasicMH()

We will be using a local VLLM instance to run the initial benchmarks and data collection.

The first model to run is the 8B model in order to collect a baseline of performance.

You can run the local VLLM instance with the following command:

Make sure to set your HF_TOKEN and HF_HOME environment variables

For Anyscale, putting models into /mnt/local_storage is a typical pattern.


`vllm serve meta-llama/Meta-Llama-3.1-8B-Instruct --port 8000 --pipeline_parallel_size 4 --enable_prefix_caching`

Lets break down what this command does:
- `vllm serve` is the command to run the VLLM server
- `meta-llama/Meta-Llama-3.1-8B-Instruct` is the model to run
- `--port 8000` is the port to run the server on
- `--pipeline_parallel_size 4` is the number of pipeline parallel size to run the server with. We are using 4 because we have 4 GPUs all of which can hold an instance of the model.
- `--enable_prefix_caching` is the flag to enable the prefix caching. This will store and reuse the beginnings of prompts to avoid repeating the same computation. This is especially useful for DSPy since we are almost always using prompts with the same beginning parts in the form of few shot demonstrations.

In [ ]:
# Command for easy copying: 
# `vllm serve meta-llama/Meta-Llama-3.1-8B-Instruct --port 8000 --pipeline_parallel_size 4 --enable_prefix_caching`
input("Press Enter once you have the vllm server running...")

In [ ]:
# TODO: switch to local model
llama_8b = dspy.MultiOpenAI(model="meta-llama/Meta-Llama-3.1-8B-Instruct", **MODEL_PARAMETERS, **LOCAL_API_PARAMETERS)

In [ ]:
# Quick sanity check to see if the program is working
with dspy.context(lm=llama_8b, rm=retriever):
    test_predictor = BasicMH()
    print(test_predictor(question="What is the capital of France?").answer)

In [ ]:
with dspy.context(lm=llama_8b, rm=retriever):
  print("Evaluating the vanilla program on the devset using the model to be trained (llama 8B)...")
  vanilla_8b_base_eval = evaluate_devset(vanilla_program)

# Running the 70B Model

Now that we have a baseline for the 8B model, let's run the 70B model and compare its performance.

## Preparation

Before running the 70B model:
1. Kill the 8B server (use `Ctrl+C`) to free up memory.
2. Remember to set your HF_TOKEN and HF_HOME environment variables
3. Use the following command to start the 70B server:

   ```
   vllm serve meta-llama/Meta-Llama-3.1-70B-Instruct --port 8000 --pipeline_parallel_size 2 --enable_prefix_caching --tensor_parallel_size 2
   ```

## Parallelism Configuration

We've chosen pipeline parallelism and tensor parallelism of 2 for the 70B model based on our current setup. Here's the reasoning:

1. Model size: The 70B model has 30 parts of ~5 GB each (based on [HuggingFace documentation](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct/tree/main)).
   - Total size: 30 * 5 GB = 150 GB

2. Available VRAM:
   - Our GPUs: 80 GB VRAM x 4 = 320 GB
   - Tensor parallelism: floor(320/150) = 2
   - Pipeline parallelism: floor(num_gpus/2) = 2
   - To use all 4 GPUs efficiently:
     - Pipeline parallel size: 2
     - Tensor parallelism: 2

3. Alternative setup (8x24GB GPUs):
   - Pipeline parallel size: 1
   - Tensor parallelism: ceil(150/24) = 7

This configuration allows us to run the 70B model efficiently across our available GPU resources.

In [ ]:
# Command for easy copying: 
# `vllm serve meta-llama/Meta-Llama-3.1-70B-Instruct --port 8000 --pipeline_parallel_size 2 --enable_prefix_caching --tensor_parallel_size 2`
input("Press Enter once you have the vllm server running...")

In [ ]:
llama_70b = dspy.MultiOpenAI(model="meta-llama/Meta-Llama-3.1-70B-Instruct", **MODEL_PARAMETERS, **LOCAL_API_PARAMETERS)

In [ ]:
# Another sanity check
with dspy.context(lm=llama_70b, rm=retriever):
    test_predictor = BasicMH()
    print(test_predictor(question="What is the capital of France?").answer)

In [ ]:
with dspy.context(lm=llama_70b, rm=retriever):
  print("Evaluating the vanilla program on the devset using llama 70B...")
  llama_70b_base_eval = evaluate_devset(vanilla_program)

We hope to bring the 8B performance up to at least 70B level

## Optimizing the LLaMa 70B pipeline

Now we are ready to optimize the pipeline. We want to optimize the 70B pipeline in order to get the best possible data to then train our 8B model.

We will use Bootstrap Few Shot with Random Search (BFRS) to optimize the pipeline.

The essence of BFRS is to try out different configurations of few shot demonstrations per step and see which one works best on the validation set.

The cool part about BFRs is that it will automatically collect the "good" chains of thought for us and add them to the examples at each step.

Now we know how well the base pipeline performs, let's run prompt optimization on the pipeline in order to juice up the performance.

Let's go over what the hyperparameters mean:
- MAX_BOOTSTRAPPED_DEMOS: DSPy will "bootstrap" the program by collecting examples at each step that are successful and reusing those in the pipeline. This means that it will automatically collect and add chains of thought to the pipeline.
- MAX_LABELED_DEMOS: DSPy will also insert some labeled demonstrations from the training set. These would be unmodified examples from the training set that are just using the gold answer.
- NUM_CANDIDATE_PROGRAMS: This is the number of candidate programs that the optimizer will generate. The actual number of programs that are created is this plus three, as DSPy will also try a program with no examples, a program with TODO (check)
- OPTIMIZER_NUM_TRAIN and OPTIMIZER_NUM_VAL: These are the number of examples that the optimizer will use for training and validation. Note that we will be taking the "validation" set from the trainset so as the actual validation set is untouched.

In [ ]:
# Optimization hyperparameters
from dspy.teleprompt.random_search import BootstrapFewShotWithRandomSearch

# Define the hyperparameters for prompt optimization
MAX_BOOTSTRAPPED_DEMOS = 3
MAX_LABELED_DEMOS = 3
NUM_CANDIDATE_PROGRAMS = 6
OPTIMIZER_NUM_TRAIN = 100
OPTIMIZER_NUM_VAL = 150

In [12]:
# Prepare the training and validation sets for the optimizer using the original
# trainset. This ensures that our actual devset is left untouched.
shuffled_trainset = [d for d in trainset]
rng.shuffle(shuffled_trainset)
optimizer_trainset = shuffled_trainset[:OPTIMIZER_NUM_TRAIN]
optimizer_valset = shuffled_trainset[OPTIMIZER_NUM_TRAIN:OPTIMIZER_NUM_TRAIN+OPTIMIZER_NUM_VAL]

NameError: name 'OPTIMIZER_NUM_TRAIN' is not defined

In [ ]:
# Initialize the optimizer
bfrs_optimizer = BootstrapFewShotWithRandomSearch(
    metric=metric,
    max_bootstrapped_demos=MAX_BOOTSTRAPPED_DEMOS,
    max_labeled_demos=MAX_LABELED_DEMOS,
    num_candidate_programs=NUM_CANDIDATE_PROGRAMS,
    num_threads=NUM_THREADS
)

# We have added this flag to save you some compute and time while running the notebook
COMPILE_PROGRAM = False

# Compile the optimizer and evaluate
with dspy.context(lm=llama_70b, rm=retriever):
    vanilla_program = BasicMH()
    if COMPILE_PROGRAM:
        bfrs_base_program = bfrs_optimizer.compile(vanilla_program, trainset=optimizer_trainset, valset=optimizer_valset)
        bfrs_base_program.save(f"basicmh_70b_bfrs_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}_{NUM_CANDIDATE_PROGRAMS}.json")
    else:
        bfrs_base_program = BasicMH()
        bfrs_base_program.load(f"basicmh_70b_bfrs_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}_{NUM_CANDIDATE_PROGRAMS}.json")
    
    llama_70b_bfrs_eval = evaluate_devset(bfrs_base_program)

In [ ]:
from dspy.teleprompt import MIPROv2

eval_kwargs = dict(display_progress=True, display_table=0, num_threads=NUM_THREADS)
teleprompter = MIPROv2(prompt_model=llama_70b, task_model=llama_70b, metric=metric, num_candidates=10, init_temperature=0.9, verbose=True)

COMPILE_PROGRAM = False
if COMPILE_PROGRAM:
    with dspy.context(lm=llama_70b, rm=retriever):
        compiled_program = teleprompter.compile(vanilla_program, trainset=optimizer_trainset, valset=optimizer_valset, num_batches=30, max_bootstrapped_demos=MAX_BOOTSTRAPPED_DEMOS,max_labeled_demos=MAX_LABELED_DEMOS, eval_kwargs=eval_kwargs, requires_permission_to_run=False)
        compiled_program.save(f"basicmh_70b_MIPROv2_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}.json")
else:
    compiled_program = BasicMH()
    compiled_program.load(f"basicmh_70b_MIPROv2_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}.json")

with dspy.context(lm=llama_70b, rm=retriever):
    llama_70b_mipro_eval = evaluate_devset(compiled_program)

### Bootstrap Data


In this section, we bootstrap data for fine-tuning. In the code block below, we are deciding which program should be used to collect the bootstraps. We are setting this to the prompt optimized program, but one could also set this to the vanilla program, though doing so would lead to lower quality bootstraps.

In [ ]:
bootstrap_program = bfrs_base_program

We do something that kind of looks like rejection sampling here. For 3 rounds, we will run the model on the dataset and collect any 
examples that are solved. We then remove the solved examples from the dataset and repeat.

In the [Large Language Monkeys paper](https://arxiv.org/pdf/2407.21787), they show that sampling up to 100 times can still get you a single correct answe in domains with ground truth or a verifier, so we can get away with this form of rejection sampling. Three rounds puts us at a decent spot on the curve of sampling up to N times. If we were in a domain where getting any correct answers was extremely hard, we may consider doing more rounds of sampling, but for now 3 rounds works.

We did some experiments to see what the sampling curve looks like:

<!-- ![Sampling Curve](./sampling_curve.png) -->
<img src="./sampling_curve.png" alt="Sampling Curve" width="500">

In [ ]:
from dspy.teleprompt.finetune_teleprompter import bootstrap_data, convert_to_module_level_prompt_completion_data, bootstrap_data_for_round
import ujson

# This should be moved inside the finetune_teleprompter class
def write_data(program, data, filename):
    print("Bootstrapping and writing data to", filename)
    correct_data = []
    unsolved_examples = data.copy()
    sampling_temperature = 0.901
    sampling_temperature_delta = 0.0001
    
    for i in range(3):
        if len(unsolved_examples) == 0:
            break
        data = bootstrap_data_for_round(program, unsolved_examples, metric=metric, num_threads=NUM_THREADS, sampling_round=i, sampling_temperature=sampling_temperature, sampling_temperature_delta=sampling_temperature_delta)
        correct_data_round = [x for x in data if x["score"]]
        correct_examples_round = set([x["example"] for x in correct_data_round])
        print(f"Round {i} complete. Solved {len(correct_data_round)} of {len(unsolved_examples)} examples. {len(unsolved_examples) - len(correct_examples_round)} examples remain unsolved.")
        unsolved_examples = [x for x in unsolved_examples if x not in correct_examples_round]

        correct_data.extend(correct_data_round)
        sampling_temperature += sampling_temperature_delta
    
    # Convert the data to prompt completion format
    dataset = convert_to_module_level_prompt_completion_data(correct_data, program=program, exclude_demos=True)
    
    # Format the data for finetuning using the LM
    print("Writing dataset with length", len(dataset), "to", filename)
    with open(filename, "w") as f:
        ujson.dump(dataset, f)


TRAIN_SIZE = 1500
EVAL_SIZE = int(TRAIN_SIZE/4)

dataset_filenames = {f"trainset_data_{TRAIN_SIZE}.json": shuffled_trainset[:TRAIN_SIZE], f"trainset_val_data_{EVAL_SIZE}.json": shuffled_trainset[TRAIN_SIZE:TRAIN_SIZE+EVAL_SIZE]}

dspy.settings.configure(experimental=True, lm=llama_70b, rm=retriever)

for filename, data in dataset_filenames.items():
    write_data(bootstrap_program, data, filename)

In [ ]:
# Let's look at an example prompt completion pair!
data_example = dataset_filenames[f"trainset_data_{TRAIN_SIZE}.json"]
print("Example prompt:")
print(data_example[0]['prompt'])
print("-"*50)
print("Example completion:")
print(data_example[0]['completion'])
print("-"*50)

Now you should kill your 70B vllm server so that you can use your GPUs for finetuning

In [ ]:
# Press enter once you have killed the 70B vllm server
input("Press Enter once you have killed the 70B vllm server (press Ctrl+C to kill)...")

# Fine-tuning

We will use LLM Forge to fine-tune the 8B model.

In order to do this, we need to format our data into the correct format (Follows OpenAI messaging format placed in a jsonl file).

We initially saved the data into a json file in prompt-completion format.

In order to prepare for finetuning, we need to do three steps:
1. Format the data into the correct format and verify that the data is valid
2. Upload the data to GCP
3. Generate the compute configuration file

After the compute configuration file is generated, we can submit the job to LLM Forge, using either the command line or using the anyscale jobs sdk.
TODO: Add the anyscale jobs sdk submit method

Be sure to checkout the fine-tuning documentation for the latest on how to use our [API](https://docs.anyscale.com/llms/finetuning/intro) and additional [capabilities](https://docs.anyscale.com/category/fine-tuning-beta/).

In [ ]:
student_llama_8b = dspy.TrainableAnyscale(model="meta-llama/Meta-Llama-3.1-8B-Instruct")

TODO: All this should be moved into the TrainableAnyscaleLM class. You should instead just call a finetune method with your datasets, hparams, compute config

In [ ]:
from dsp.modules.lm import TrainingMethod

train_path = f"trainset_data_{TRAIN_SIZE}.json"
eval_path = f"trainset_val_data_{EVAL_SIZE}.json"
method = TrainingMethod.SFT
kwargs = {
    "hyperparameters": {
        "num_devices": 4,
        "trainer_resources": None,
        "worker_resources": None
    },
    "use_lora": True
}

if method != TrainingMethod.SFT:
    raise NotImplementedError("Only SFT training is supported at the moment.")

train_dataset = student_llama_8b._format_data_for_vanilla_finetuning(train_path)
val_dataset = student_llama_8b._format_data_for_vanilla_finetuning(eval_path) if eval_path else None

if not student_llama_8b._verify_datasets(train_dataset, val_dataset):
    print("Unable to verify arguments")
    raise RuntimeError("Unable to verify argument")

# TODO: This should be a function inside the TrainableAnyscaleLM class
formatted_paths = {}
for path, dataset in [(train_path, train_dataset), (eval_path, val_dataset)]:
    if not (path and dataset):
        continue
    formatted_path = path.split(".")[0] + "_formatted.jsonl"
    with open(formatted_path, "w") as f:
        for item in dataset:
            f.write(ujson.dumps(item) + "\n")

    formatted_paths[path] = formatted_path

remote_train_path, remote_eval_path = student_llama_8b._submit_data(train_path=formatted_paths[train_path], eval_path=formatted_paths[eval_path])
compute_config_path, compute_config = student_llama_8b._generate_config_files(train_path=remote_train_path, eval_path=remote_eval_path, **kwargs)


We'll fine-tune our LLM by choosing a set of configurations. We have created recipes for different LLMs in the [`training configs`](configs/training/lora/llama-3-8b.yaml) folder which can be used as is or modified for experiments. These configurations provide flexibility over a broad range of parameters such as model, data paths, compute to use for training, number of training epochs, how often to save checkpoints, padding, loss, etc. We also include several [DeepSpeed](https://github.com/microsoft/DeepSpeed) [configurations](configs/deepspeed/zero_3_offload_optim+param.json) to choose from for further optimizations around data/model parallelism, mixed precision, checkpointing, etc.

We also have recipes for [LoRA](https://arxiv.org/abs/2106.09685) (where we train a set of small low ranked matrices instead of the original attention and feed forward layers) or full parameter fine-tuning. We recommend starting with LoRA as it's less resource intensive and quicker to train.

In [ ]:
# View the compute config
os.system(f"cat {compute_config_path}")

In [ ]:
# TODO: Get job working with LLMForge
# job_id: str = anyscale.job.submit(
#     compute_config
# )
# anyscale.job.wait(id=job_id, timeout_s=18000)
# print(f"Job {job_id} succeeded!")

SKIP_FT = True
if not SKIP_FT:
    command = compute_config.entrypoint
    print(command)
    os.system(command)

In [ ]:
# Lets sanity check the finetuned model. We need to download it first.
# After the finetuning is complete, the logs will say something like "The best performing weights have been forwarded to gs://..."

def download_from_gcp_bucket(bucket_name, source_folder, destination_folder):
    """Downloads a folder from a GCP bucket to a local folder.

    Args:
        bucket_name (str): The name of the GCP bucket.
        source_folder (str): The path to the folder in the bucket.
        destination_folder (str): The local path where files should be saved.

    Returns:
        str: The path to the downloaded folder.
    """
    import google.cloud.storage as storage
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=source_folder)

    for blob in blobs:
        if blob.name.endswith('/'):
            continue  # Skip directories
        
        relative_path = os.path.relpath(blob.name, source_folder)
        local_file_path = os.path.join(destination_folder, relative_path)
        
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        # only download if the file doesn't exist
        if not os.path.exists(local_file_path):
            blob.download_to_filename(local_file_path)
            print(f"Downloaded {blob.name} to {local_file_path}")

    print(f"Folder {source_folder} downloaded to {destination_folder}.")
    return destination_folder

In [ ]:
input("Press Enter once you have copied the path from the logs...")

In [ ]:
# Copy paste the path from the logs.
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
lora_source_path = "gs://storage-bucket-cld-tffbxe9ia5phqr1unxhz4f7e1e/org_4snvy99zwbmh4gbtk64jfqggmj/cld_tffbxe9ia5phqr1unxhz4f7e1e/artifact_storage/lora_fine_tuning/meta-llama/Meta-Llama-3.1-8B-Instruct:isaac:nfomi"

# # Parse the GCS path
bucket_name = lora_source_path.split('/')[2]
source_folder = '/'.join(lora_source_path.split('/')[3:])

# Download the LoRA model folder locally
local_lora_path = download_from_gcp_bucket(bucket_name, source_folder, "/mnt/local_storage/dspy/mhqa-lora")

# Evaluation

Throughout this section, anything using 8B model (or technically 70B too) should use the new evaluate with ray data batch offline(or technically online) inference.

Probably worth testing offline with 8x8 threads vs just 64 threads to see if it makes a meaningful difference.

## Performance comparisons

- 70B
- 70B BSFS
- 8B
- 8B BSFT
- 8B BSFT + BSFS

First, we need to serve the base model and tell VLLM where to find the LoRA weights

Run the following command:

```
vllm serve meta-llama/Meta-Llama-3.1-8B-Instruct --port 8000 --pipeline_parallel_size 4 --enable_prefix_caching --enable_lora --lora_modules mhqa-lora=/mnt/local_storage/dspy/mhqa-lora
```

# Explanation:
This command starts a VLLM server to serve the Meta-Llama-3-8B-Instruct model with LoRA fine-tuning.
Here's a breakdown of the command:
- 'vllm serve': Starts the VLLM server
- 'meta-llama/Meta-Llama-3.1-8B-Instruct': Specifies the base model to use
- '--port 8000': Sets the server port to 8000
- '--pipeline_parallel_size 4': Enables pipeline parallelism with 4 stages
- '--enable_prefix_caching': Enables caching of prefixes for faster inference
- '--enable_lora': Enables LoRA (Low-Rank Adaptation) for fine-tuning
- '--lora_modules mhqa-lora=/mnt/local_storage/dspy/mhqa-lora': Specifies the name of the LoRA module and the path to the LoRA weights. We use the name instead of the base model name when trying to use the LoRA weights. If we just use the base model name, the server will ignore the LoRA weights.

This setup allows us to serve a fine-tuned version of the 8B model, which we'll use for subsequent evaluations.

In [ ]:
# Command for easy copying: 
# `vllm serve meta-llama/Meta-Llama-3.1-8B-Instruct --port 8000 --pipeline_parallel_size 4 --enable_prefix_caching --enable_lora --lora_modules mhqa-lora=/mnt/local_storage/dspy/mhqa-lora`

llama_8b = dspy.MultiOpenAI(model="meta-llama/Meta-Llama-3.1-8B-Instruct", **LOCAL_API_PARAMETERS, **MODEL_PARAMETERS)
mhqa_llama_8b = dspy.MultiOpenAI(model="mhqa-lora", **LOCAL_API_PARAMETERS, **MODEL_PARAMETERS)

In [ ]:
with dspy.context(lm=mhqa_llama_8b, rm=retriever):
    vanilla_program = BasicMH()
    llama_8b_finetuned_eval = evaluate_devset(vanilla_program)

Now let's try optimizing the program with the finetuned model

In [ ]:
COMPILE_PROGRAM = False

with dspy.context(lm=mhqa_llama_8b, rm=retriever):
    vanilla_program = BasicMH()
    if COMPILE_PROGRAM:
        bfrs_finetuned_program = bfrs_optimizer.compile(vanilla_program, trainset=optimizer_trainset, valset=optimizer_valset)
        bfrs_finetuned_program.save(f"basicmh_8b_ft_bfrs_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}_{NUM_CANDIDATE_PROGRAMS}.json")
    else:
        bfrs_finetuned_program = BasicMH()
        bfrs_finetuned_program.load(f"basicmh_8b_ft_bfrs_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}_{NUM_CANDIDATE_PROGRAMS}.json")
    llama_8b_bfrs_finetuned_eval = evaluate_devset(bfrs_finetuned_program)

In [68]:
COMPILE_PROGRAM = True
with dspy.context(lm=mhqa_llama_8b, rm=retriever):
    vanilla_program = BasicMH()
    if COMPILE_PROGRAM:
        teleprompter = MIPROv2(prompt_model=llama_8b, task_model=mhqa_llama_8b, metric=metric, num_candidates=10, init_temperature=0.9, verbose=True)
        compiled_program = teleprompter.compile(vanilla_program, trainset=optimizer_trainset, valset=optimizer_valset, num_batches=30, max_bootstrapped_demos=MAX_BOOTSTRAPPED_DEMOS,max_labeled_demos=MAX_LABELED_DEMOS, eval_kwargs=eval_kwargs, requires_permission_to_run=False)
        compiled_program.save(f"basicmh_8b_ft_MIPROv2_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}.json")
    else:
        compiled_program = BasicMH()
        compiled_program.load(f"basicmh_8b_ft_MIPROv2_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}.json")
    llama_8b_ft_mipro_eval = evaluate_devset(compiled_program)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 3 lm calls in program + (4) lm calls in program aware proposer = 44 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 100 examples in train set * 3 full evals = 1050 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may

b: 10
b: 20
b: 30
b: 40
b: 50
b: 60
b: 70
b: 80
b: 90
summary: Prediction(
    summary="The dataset appears to be a comprehensive collection of trivia questions, presenting a diverse range of topics, including entertainment, history, arts, and culture. Most questions require specific, factual answers, often testing users' recall of specific details or events, and are designed to test users' knowledge on subject matter. The syntax and length of the questions vary, but most are concise and require precise answers."
)
DATA SUMMARY: The dataset appears to be a comprehensive collection of trivia questions, presenting a diverse range of topics, including entertainment, history, arts, and culture. Most questions require specific, factual answers, often testing users' recall of specific details or events, and are designed to test users' knowledge on subject matter. The syntax and length of the questions vary, but most are concise and require precise answers.


  9%|▉         | 9/100 [01:04<10:56,  7.22s/it]


Bootstrapped 3 full traces after 10 examples in round 0.


  4%|▍         | 4/100 [00:18<07:18,  4.57s/it]


...

Average Metric: 0 / 1  (0.0): 100%|██████████| 1/1 [00:05<00:00,  5.32s/it]





Given the context and question, generate a specific search query that would lead to relevant passages for answering the question, and provide a detailed reasoning for this search query.

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the output fields}. We ...

Search Query: ${search_query}

---

Context: N/A

Question: When was the 1st Earl of Ypres born who attacked attacked the Boer force from the rear while mounted infantry?

Please provide the output fields Reasoning then Search Query. Do so immediately, without additional content before or after, and precisely as the format above shows. Begin with the field Reasoning.

Reasoning: Let's think step by step in order to produce the output fields. We need to extract the key information from the question, which is the title of the person, the event, and the action. This will help us formulate a search query that can provide the answer to the qu

Average Metric: 49 / 100  (49.0): 100%|██████████| 100/100 [00:41<00:00,  2.40it/s]


UPDATING BEST SCORE WITH 49.0


  0%|          | 0/150 [00:00<?, ?it/s]

Lastly, lets give the base 8B model a fair chance by prompt optimizing it.

In [ ]:
COMPILE_PROGRAM = False

with dspy.context(lm=llama_8b, rm=retriever):
    vanilla_program = BasicMH()
    if COMPILE_PROGRAM:
        bfrs_program = bfrs_optimizer.compile(vanilla_program, trainset=optimizer_trainset, valset=optimizer_valset)
        bfrs_program.save(f"basicmh_8b_bfrs_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}_{NUM_CANDIDATE_PROGRAMS}.json")
    else:
        bfrs_program = BasicMH()
        bfrs_program.load(f"basicmh_8b_bfrs_{MAX_BOOTSTRAPPED_DEMOS}_{MAX_LABELED_DEMOS}_{NUM_CANDIDATE_PROGRAMS}.json")
    llama_8b_bfrs_eval = evaluate_devset(bfrs_program)

In [ ]:
# Now we can compare all iterations of this pipeline
print(f"Results for HotPotQA fine-tuning LLaMa 8B with a starting trainset")
print(f"    70B model (vanilla program): {llama_70b_base_eval}")
print(f"    70B model (bfrs program): {llama_70b_bfrs_eval}")
print(f"    8B model (vanilla program): {vanilla_8b_base_eval}")
print(f"    8B model (bfrs program): {llama_8b_bfrs_eval}")
print(f"    8B model (finetuned program): {llama_8b_finetuned_eval}")
print(f"    8B model (finetuned bfrs program): {llama_8b_bfrs_finetuned_eval}")
print(f"    8B model (finetuned mipro program): {llama_8b_ft_mipro_eval}")

TODO: Let's now use the new offline batch inference to evaluate the finetuned model with optimized program on the entire devset

In [ ]:
# TODO: implement once done

In [ ]:
raise NotImplementedError("Stop here")

# Serving

This is the second biggest unknown
I imagine it to be easy, but crazier things have happened

I need to keep a reference or link to the LLM forge job inside the LM.finetune method

how do I get the ray llm image!

We'll start by running the rayllm CLI command below to start the workflow to generate the service yaml configuration:
```bash
mkdir /home/ray/default/deploy/services
cd /home/ray/default/deploy/services
rayllm gen-config 
```

<img src="assets/cli.png" width=500 alt="todo! get this inage of what I need to serve">


## Batch offline inference
- Compare running inference using 
    - Ray Data 
    - multithreading on local VLLM thru HTTP
    - Multithreading to Ray Serve instance thru HTTP
- Dev time estimate: 7 days

<b style="background-color: yellow;">&nbsp;🛑 IMPORTANT&nbsp;</b>: Please `Terminate` your service from the Service page to avoid depleting your free trial credits.

In [ ]:
# Clean up
!python src/clear_cell_nums.py
!find . | grep -E ".ipynb_checkpoints" | xargs rm -rf
!find . | grep -E "(__pycache__|\.pyc|\.pyo)" | xargs rm -rf
!rm -rf __pycache__ data .HF_TOKEN deploy/services